In [1]:
from timeit import default_timer

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

from numpy import ceil
import pandas

from riptide import TimeSeries, ffa_search, generate_signal, ffa_transform_2d, downsample
from riptide.libffa import get_snr

%load_ext autoreload
%autoreload 2

Data we want:
- name
- coordinates
- dm
- start_time
- frequency band (low, high, df)
- tsamp
- nsamp


In [2]:
def fold(tseries, period, bins=250):
    bins = int(bins)
    period = float(period)
    dsfactor = (period / bins) / tseries.tsamp
    ts = tseries.downsample(dsfactor)
    npulses = ts.nsamp // bins
    return ts.data[:npulses*bins].reshape(npulses, bins)

In [7]:
#data = np.load('/home/vmorello/work/pulsar_raw_data/J1914+0845.64us.npy')
#data = np.fromfile('/home/vmorello/work/presto_datfiles/J1921-0459_DM100.00.dat', dtype=np.float32)
#time_series = TimeSeries.from_npy('/home/vmorello/work/pulsar_raw_data/J1914+0845.64us.npy', tsamp=64e-6)
#time_series = TimeSeries.from_presto_inf('/home/vmorello/work/slowpoke/guppi_57956_Cent_0001_0001_DM950.00.inf')


# nsamp = 2**23
# tsamp = 64e-6
# period = 564.788e-3
# time_series = TimeSeries(
#     generate_signal(nsamp, period/tsamp, phi0=0.5, ducy=0.04), tsamp
#     )

time_series = TimeSeries.from_presto_inf('/home/vince/work/pulsars/presto_time_series/J1855+0307_DM400.00.inf')
#time_series.data[:500] += 40.0
print(time_series)

TimeSeries {nsamp = 8754411, tsamp = 6.4000e-05, tobs = 560.282}


In [17]:
start = default_timer()
ts, plan, pgram = ffa_search(time_series, period_min=0.5, period_max=60.0, bins_min=480, bins_max=520, threads=4, rmed_width=4.0)
end = default_timer()
print(end - start)
#pgram.display()

1.0544979180049268


In [18]:
pgram.periods.size

553543

In [11]:
time_series = TimeSeries.from_presto_inf('/home/vince/work/pulsars/presto_time_series/J1855+0307_DM400.00.inf')

ts, plan, pgram = ffa_search(time_series, period_min=0.4, period_max=1.8, bins_min=480, bins_max=520, threads=4)

pgram.display()

In [55]:
plt.figure(figsize=(18, 5))
plt.plot(pgram.periods, pgram.snrs.max(1), marker='o', markersize=2, alpha=0.5)
plt.xlim(0.4, pgram.periods[-1])
plt.grid()
plt.xlabel('Trial Period (s)', fontsize=20)
plt.ylabel('Best S/N', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.show()

In [7]:
plan

Number of samples  : 8754411
Sampling time      : 6.400000e-05

Initial downsampling factor: 26.041667
Num. samples after initial downsampling : 336169
Sampling time after initial downsampling: 1.666667e-03

Width trials (bins): 1, 2, 3, 4, 6, 9, 13, 19, 28, 42

    dsfactor     tsamp  bins_min  bins_max  period_min  period_max
0   1.000000  0.001667       240       260    0.400000    0.433333
1   1.083333  0.001806       240       260    0.433333    0.469444
2   1.173611  0.001956       240       260    0.469444    0.508565
3   1.271412  0.002119       240       260    0.508565    0.550945
4   1.377363  0.002296       240       260    0.550945    0.596857
5   1.492143  0.002487       240       260    0.596857    0.646595
6   1.616489  0.002694       240       260    0.646595    0.700478
7   1.751196  0.002919       240       260    0.700478    0.758852
8   1.897129  0.003162       240       260    0.758852    0.822089
9   2.055223  0.003425       240       260    0.822089    0.890597


In [39]:
bins = 4000
subints = fold(time_series.deredden(8.0), 12.1216, bins=bins)
#transform = ffa_transform_2d(subints)

#subints = subints.reshape(-1, 100, bins).sum(axis=1)

plt.figure(figsize=(12, 8), dpi=200)
plt.subplot(211)
plt.imshow(subints, cmap=plt.cm.Greys, aspect='auto')

plt.subplot(212)
plt.plot(subints.sum(0))
plt.xlim(0, bins)
plt.tight_layout()
plt.show()

In [32]:
plt.figure(figsize=(24, 10), dpi=150)

plt.plot(subints.var(0))

plt.xlim(0, bins)
plt.tight_layout()
plt.show()

In [61]:
subints.shape #.reshape(-1, 68, 512).sum(1).shape

(4555, 240)

In [35]:
from riptide.ffautils import get_period

# def num_trials(plan):
#     num = 0
#     for istep, step in plan.steps.iterrows():
#         ds = step.dsfactor
#         ns = int(plan.nsamp / ds)
#         for b in range(int(step.bins_min), int(step.bins_max)):
#             num += ns // b
#     return num


def num_trials(plan):
    num = 0
    for istep, step in plan.steps.iterrows():
        ds = step.dsfactor
        ns = int(plan.nsamp / ds)
        b = np.arange(int(step.bins_min), int(step.bins_max))
        num += (ns // b).sum()
    return num


def period_trials(plan):
    num = 0
    b = []
    m = []
    for istep, step in plan.steps.iterrows():
        ds = step.dsfactor
        ns = int(plan.nsamp / ds)
        bins = np.arange(int(step.bins_min), int(step.bins_max))
        b.append(bins)
        m.append(ns // bins)
    return np.concatenate(m), np.concatenate(b)


def period_trials(plan):
    periods = []
    for istep, step in plan.steps.iterrows():
        ds = step.dsfactor
        ns = int(plan.nsamp / ds)
        bins_range = np.arange(int(step.bins_min), int(step.bins_max))
        nprofs = ns // bins_range
        
        for m, b in zip(nprofs, bins_range):
            periods.append( ds * get_period(np.arange(m), m, b) )
            
    return np.concatenate(periods)
            
        
# Input arrays 
dsfactors = plan.steps.dsfactor.as_matrix()
bins_min = plan.steps.bins_min.as_matrix()
bins_max = plan.steps.bins_max.as_matrix()

# Output arrays
nt = num_trials(plan)
threads = 1
periods = np.zeros(nt, dtype=np.float32)
snrs = np.zeros(nt * plan.widths.size, dtype=np.float32)

print('num_trials =', nt)

LibFFA.py_periodogram(
    data, data.size, 
    dsfactors, bins_min, bins_max, dsfactors.size, 
    plan.widths, plan.widths.size,
    threads,
    periods,
    snrs
    )

snrs = snrs.reshape(nt, plan.widths.size)

num_trials = 396118


In [36]:
plt.plot(periods * tsamp, snrs.max(axis=1), 'k.', markersize=1)
plt.grid()

In [6]:
snrs.shape

(722839, 17)

In [33]:
timeit LibFFA.py_periodogram(data, data.size, dsfactors, bins_min, bins_max, dsfactors.size, plan.widths, plan.widths.size, threads, periods, snrs)

1 loop, best of 3: 972 ms per loop


In [43]:
timeit num_trials(plan)

100 loops, best of 3: 3.56 ms per loop


In [57]:
ms, bs = period_trials(plan)

In [61]:
periods = np.zeros(nt)
ix = 0
for m, b in zip(ms, bs):
    periods[ix:ix+m] = get_period(np.arange(m), m, b)
    ix += m
    

In [66]:
timeit period_trials(plan)

10 loops, best of 3: 16 ms per loop


In [72]:
plt.plot(period_trials(plan))

In [71]:
np.diff(period_trials(plan)).argmin()

245928

In [29]:
plan.steps

,dsfactor,tsamp,bins_min,bins_max,period_min,period_max
0,32.894737,0.002105,190,210,0.400000,0.442105
1,36.357341,0.002327,190,210,0.442105,0.488643
2,40.184429,0.002572,190,210,0.488643,0.540079
3,44.414369,0.002843,190,210,0.540079,0.596929
4,49.089566,0.003142,190,210,0.596929,0.659764
5,54.256889,0.003472,190,210,0.659764,0.729213
6,59.968140,0.003838,190,210,0.729213,0.805972
7,66.280576,0.004242,190,210,0.805972,0.890811
8,73.257479,0.004688,190,210,0.890811,0.984581
9,80.968792,0.005182,190,210,0.984581,1.088221


In [18]:
wdata = fft_deredden(data, tsamp)

In [116]:
plt.plot(tseries.data, 'k.', markersize=1)

0.17150381